# CBS for MAPF
<center><img src="img/1.png"/></center>

# Preparing Data

Сначала надо превратить все тесты в формат .xml 

In [1]:
%pip install dicttoxml
import os
from dataclasses import dataclass, asdict, field
import xml.etree.ElementTree as ET

from dicttoxml import dicttoxml
import pandas as pd
from tqdm import tqdm 

Note: you may need to restart the kernel to use updated packages.


In [13]:
@dataclass
class Map:
    map_file_path: str
    grid: list[list[str]] = field(init=False, default_factory=list)
    width: int = field(init=False)
    height: int = field(init=False)
    name: str = field(init=False)

    def __post_init__(self):
        with open(self.map_file_path) as f:
            f.readline() # skipping map type
            _, height = f.readline().split()
            self.height = int(height)
            _, width = f.readline().split()
            self.width = int(width)
            f.readline() # skipping map name

            for line in f.readlines():
                self.grid.append([ str(int(s in ('@', 'T'))) for s in line ])
        self.name = self.map_file_path.split('/')[-1].replace('.', '_')

    @property
    def grid_as_str(self) -> list[str]:
        return [' '.join(self.grid[h]) for h in range(self.height)]


@dataclass
class Scenario:
    scen_file_path: str = field(repr=False)
    tasks_count: int = 1
    maxtime: int = 1000
    single_execution: bool = False
    starts: list[tuple[int, int]] = field(init=False, default_factory=list)
    goals: list[tuple[int, int]] = field(init=False, default_factory=list)
    agents_count: int = field(init=False, default=0)
    starting_agent_number: int = 1
    log_filename: str = None
    log_filepath: str = 'log'

    def __post_init__(self):
        with open(self.scen_file_path) as f:
            f.readline() # skipping task version

            for line in f.readlines():
                _, _, _, _, start_i, start_j, goal_i, goal_j, _ = line.split()
                self.starts.append((start_i, start_j))
                self.goals.append((goal_i, goal_j))
                
            self.agents_count = len(self.starts)

    @property
    def options(self):
        return {
            'tasks_count': str(self.tasks_count),
            'maxtime': str(self.maxtime),
            'single_execution': str(self.single_execution).lower()
        }



@dataclass
class Algorithm:
    planner: str = 'cbs'
    low_level: str = 'astar'
    with_cat: bool = True
    with_perfect_h: bool = False
    with_card_conf: bool = True
    with_disjoint_splitting: bool = False
    focal_w: float = 1.0

    def todict(self):
        return {k: str(v).lower() for k, v in asdict(self).items()}
        

@dataclass 
class Config:
    map: Map
    algorithm: Algorithm
    scenario: Scenario

In [3]:
def map_to_xml(map: Map, algorithm: Algorithm, scen: Scenario, map_filename: str, agents_filename):
    root = ET.Element('root')

    # grid-related stuff
    map_ = ET.SubElement(root, 'map')
    
    grid = ET.SubElement(map_, 'grid')
    grid.attrib = {'width': str(map.width), 'height': str(map.height)}

    for h in range(map.height):
        row = ET.SubElement(grid, 'row')
        row.text = map.grid_as_str[h]

    # algorithm-related stuff
    algorithm = dicttoxml(algorithm.todict(), custom_root='algorithm', 
                          xml_declaration=False, attr_type=False)
    algorithm = ET.fromstring(algorithm)
    root.append(algorithm)


    # agents-related stuff
    options = dicttoxml(scen.options, custom_root='options', 
                          xml_declaration=False, attr_type=False)
    options = ET.fromstring(options)
    agents_file = ET.SubElement(options, 'agents_file')
    agents_file.text = agents_filename.split('.')[0]
    agents_range = ET.SubElement(options, 'agents_range')
    agents_range.attrib = {'min': str(scen.starting_agent_number), 'max': str(scen.agents_count)}
    log_file = ET.SubElement(options, 'logfilename')
    if scen.log_filename:
        log_file.text = scen.log_filename
        log_path = ET.SubElement(options, 'logpath')
        log_path.text = scen.log_filepath
    aggr_results = ET.SubElement(options, 'aggregated_results')
    aggr_results.text = 'true'

    root.append(options)

    
    # building tree and writing into file
    tree = ET.ElementTree(root)

    with open(map_filename, 'wb') as file:
        ET.indent(tree, space='\t', level=0)
        tree.write(file, xml_declaration=True)

def scenario_to_xml(scen: Scenario, filename: str):
    root = ET.Element('root')

    for agent_id in range(scen.agents_count):
        start_i, start_j = scen.starts[agent_id]
        goal_i, goal_j = scen.goals[agent_id] 
        agent = ET.SubElement(root, 'agent')
        agent.attrib = {'id': str(agent_id),
                      'start_i': start_j,
                      'start_j': start_i,
                      'goal_i': goal_j,
                      'goal_j': goal_i,
                      }

    tree = ET.ElementTree(root)

    filename = filename.split('.')[0] + f'-{scen.tasks_count}.xml'
    with open(filename, 'wb') as file:
        ET.indent(tree, space='\t', level=0)
        tree.write(file, xml_declaration=True)

def movingai_task_to_xml(config: Config, map_filename: str, scen_filename: str):
    scenario_to_xml(config.scenario, scen_filename)
    map_to_xml(config.map, config.algorithm, config.scenario, map_filename, scen_filename)

In [4]:
def xml_results_to_pd(xml_filename: str) -> pd.DataFrame:
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    agents_count = []
    success_count = []
    time = []
    flowtime = []
    makespan = []
    sic = []

    # Extracting data from XML
    for result in root.findall('.//result'):
        agents_count.append(int(result.attrib['agents_count']))
        success_count.append(int(result.attrib['success_count']) == 1)
        time.append(float(result.find('.//iteration').attrib['time']))
        flowtime.append(int(result.find('.//iteration').attrib['flowtime']))
        makespan.append(int(result.find('.//iteration').attrib['makespan']))
        sic.append(int(result.find('.//iteration').attrib['makespan']))

    return pd.DataFrame({'agents_count': agents_count, 
                         'success_count': success_count, 
                         'time': time,
                         'flowtime': flowtime,
                         'makespan': makespan,
                         'SIC': sic}).set_index('agents_count')    

In [ ]:
for m in ['empty-48-48.map', 'Paris_1_256.map', 'room-64-64-16.map', 'warehouse-20-40-10-2-2.map']:
    map_g = Map(m)
    print(map_g.name, map_g.width, map_g.height, sum([sum(map(int, l)) for l in map_g.grid])/( map_g.width * map_g.height))

empty-48-48_map 48 48 2304 0.0
Paris_1_256_map 256 256 65536 0.2791748046875
room-64-64-16_map 64 64 4096 0.10986328125
warehouse-20-40-10-2-2_map 340 164 55760 0.30494978479196555


# Becnhmarking

In [15]:
MAP_PATHS = ['empty-48-48.map', 'Paris_1_256.map', 'room-64-64-16.map', 'warehouse-20-40-10-2-2.map']
SCEN_PATHS = ['scen-empty/empty-48-48-even-2.scen', 'scen-paris/Paris_1_256-even-3.scen',
              'scen-room/room-64-64-16-even-9.scen', 'scen-warehouse/warehouse-20-40-10-2-2-even-11.scen']

@dataclass
class AlgorithmSpecification:
    name: str
    params: dict

CBS_ALGORITHM = AlgorithmSpecification('cbs', {'planner': 'cbs', 'with_perfect_h': True})
CBS_DS_ALGORITHM = AlgorithmSpecification('cbs_ds', {'planner': 'cbs', 'with_perfect_h': True, 'with_disjoint_splitting': True})
ECBS_30_ALGORITHM = AlgorithmSpecification('ecbs_30', {'planner': 'ecbs', 'with_perfect_h': True, 'low_level': 'focal_search', 'focal_w': 3.0})
ECBS_179_ALGORITHM = AlgorithmSpecification('ecbs_179', {'planner': 'ecbs', 'with_perfect_h': True, 'low_level': 'focal_search', 'focal_w': 1.79})
ALG_PARAMS = [CBS_ALGORITHM, CBS_DS_ALGORITHM, ECBS_30_ALGORITHM, ECBS_179_ALGORITHM]

In [16]:
def get_map_result(alg: AlgorithmSpecification, map: Map, scen: Scenario) -> pd.DataFrame:
    alg_name, alg_params = alg.name, alg.params
    scen.agents_count = min(scen.agents_count, 300)
    conf = Config(map, Algorithm(**alg_params), scen)
    movingai_task_to_xml(conf, 'testmap.xml', 'testmap-agents.xml')
    status = os.system('../source/CBS testmap.xml >/dev/null 2>&1')
    if not status:
        return xml_results_to_pd('testmap_log.xml').add_suffix('_' + alg_name)
    else:
        raise RuntimeError('C++ code failed with error.')

def benchmarking(map_paths: list[str], 
                 scen_paths: list[str], 
                 algorithms_params: list[AlgorithmSpecification],
                 max_time: int = 1000) -> list[tuple[str, pd.DataFrame]]:
    results = []

    for map, scen in zip(map_paths, scen_paths):
        df_map = pd.DataFrame(columns=["agents_count"]).set_index("agents_count")
        cur_map = Map(map_file_path=map)
        cur_scen = Scenario(scen_file_path=scen, 
                maxtime=max_time)
        for alg in tqdm(algorithms_params, desc=f"Running algorithms on {cur_map.map_file_path}"):
            df_map = pd.concat([df_map, get_map_result(alg, cur_map, cur_scen)], axis=1, join='outer')
        
        results.append((cur_map.name, df_map))

    return results

In [7]:
res = benchmarking(MAP_PATHS, SCEN_PATHS, ALG_PARAMS, max_time=60*1000)

Running algorithms on data/room-64-64-16.map: 100%|██████████| 4/4 [1:17:51<00:00, 1167.84s/it]
Running algorithms on data/warehouse-20-40-10-2-2.map: 100%|██████████| 4/4 [4:47:23<00:00, 4310.78s/it]  


In [8]:
for name, r in res:
    r.to_csv(name + '.csv')

Данные готовы, теперь перейдем к их изучению!